In [ ]:
#author: Mathieu Hénault
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from Bio import SeqIO
from Bio import SeqRecord
from scipy.spatial.distance import cdist
from scipy.stats import mannwhitneyu
import itertools
import seaborn as sns
import networkx as nx
import statsmodels.api as sm
import re
from collections import Counter
import pickle as pkl
import gzip

In [ ]:
sns.set(style='ticks', font='DejaVu Sans')

In [ ]:
# import assemblies 

FASTA={}
fasta={}

fasta_temp=[]
with open('/Users/mathieu/mhenault_landrylab/Sequences/ref_genomes/S288C_pacbio/S288c.genome.fa') as file_in:
    for j in SeqIO.parse(file_in, "fasta"):
        fasta_temp.append(j)
# whole-genome concatenated sequence
FASTA="".join([str(j.seq) for j in fasta_temp])
# dictionary of start coordinates for each contig
tig_starts=[0]+list(np.cumsum([len(j.seq) for j in fasta_temp]))[:-1]
tig_names=[j.id for j in fasta_temp]
fasta = pd.Series(tig_starts, index=tig_names).sort_values()
tig_order = pd.Series(range(16), index=tig_names)

In [ ]:
strains = ["barcode11.ont", 'barcode11_sub.ont', "A2565.pb", "T58.pb", 's288c.pb', 's288c_translocation.pb']
# input the total bases mapped for each strain
cov = dict(zip(strains,
               np.array([709805531, 113027745, 105848406, 139709850, 674174505, 933961654])/12e6))
# strains aliases 
strains_alias = dict(zip(strains, ["Jean-Talon","Jean-Talon","A.2565","A.T-58","S288C","S288C_translocation"]))
strains_alias = {s:strains_alias[s]+f'\n{v:.0f}X' for s,v in cov.items()}
strains_svim = strains[:5]

In [ ]:
# Import the VCF files

In [ ]:
VCF={}
for s in strains_svim:
    with open("/Volumes/MacintoshHD/Dropbox/Jean_Talon/svim/"+s+"/final_results.vcf", "r") as fi:
        vcf = [line.split("\t") for line in fi.read().splitlines() if line[0]!="#"]
        VCF[s] = pd.DataFrame(vcf).astype({1:int, 5:int})
        # filter the types of structural variants
        VCF[s] = VCF[s].loc[
            (VCF[s][4].isin(['<DEL>','<DUP:TANDEM>','<DUP_INT>','<INS>','<INV>']))
        ]

        for f in ["pos_abs","SVTYPE", "END", "SVLEN", "SUPPORT", "STD_SPAN", "STD_POS"]:
            VCF[s][f] = np.repeat(0, VCF[s].shape[0])
        # get absolute coordinate for variants
        for i in VCF[s].index:
            chrom = VCF[s].loc[i, 0]
            pos = VCF[s].loc[i, 1]
            VCF[s].loc[i, "pos_abs"] = fasta[chrom]+int(pos)
            
            for j in VCF[s].loc[i, 7].split(";"):
                j_split = j.split("=")
                if len(j_split)==2:
                    VCF[s].loc[i, j_split[0]] = j_split[1]
        # convert coordinates into rad for circular maps
        VCF[s]["pos_rad"] = VCF[s]["pos_abs"]/len(FASTA)*2*np.pi
        VCF[s]["strain"] = s
        VCF[s] = VCF[s].astype({"END":int, "SVLEN":int, "SUPPORT":int})

In [ ]:
SV = {}
for s in strains_svim:
    threshold = np.ceil(0.15*cov[s])
    # filter out the hits that have a score below 15% of the coverage depth
    SV[s]=VCF[s].copy().drop(VCF[s].loc[VCF[s][5]<threshold].index, axis=0)
SV = pd.concat(SV.values(), axis=0)

In [ ]:
# circular map of the predicted structural variants
color_sv = {"DEL":"red", "INS":"green", "DUP:TANDEM":"deepskyblue", "DUP_INT":"navy", "INV":"fuchsia"}
alias_sv = {"DEL":"Deletions", "INS":"Insertions", "DUP:TANDEM":"Tandem\nduplications", "DUP_INT":"Interspersed\nduplications", "INV":"Inversions"}
shift_sv = {"DEL":0, "INS":0.12, "DUP:TANDEM":0.24, "DUP_INT":0.36, "INV":0.48}
strains_svim = strains[:5]
y_pos_dict = dict(zip(strains_svim, range(5,10)))

fig, ax = plt.subplots(figsize=[10,10], subplot_kw={"projection":"polar"},
                      gridspec_kw={'left':0.02, 'bottom':0.02, 'right':0.98, 'top':0.98})

ax.set_theta_zero_location("N")
ax.set_theta_direction("clockwise")
ax.grid(False)
ax.axis("off")

xshrink = 0.9 # leave space at rad=0 for strain IDs
yscale = 10

tig_bound = sorted(list(fasta.values)+[len(FASTA)])
for i in range(len(tig_bound)):
    if (i%2 == 0) and (i!=16):
        pos1 = tig_bound[i]/len(FASTA)*2*np.pi*xshrink
        pos2 = tig_bound[i+1]/len(FASTA)*2*np.pi*xshrink
        for j in np.arange(pos1, pos2, 0.01):
            ax.plot([j, j], [4.5,9.5], color="0.9", linewidth=5)
    # plot the chromosome IDs
    if i!=16:
        pos = np.mean([tig_bound[i], tig_bound[i+1]])/len(FASTA)*2*np.pi*xshrink
        ax.text(pos, 3.8, fasta.index[i], va="center", ha="center", size=10)
        
            
# plot histograms of counts of SVs in 10kb windows
for (s,t), df in SV.groupby(['strain','SVTYPE']):
    y_pos = y_pos_dict[s]
    ax.text(-0.03, y_pos, strains_alias[s], ha="right", va="center", size=8)

    color = color_sv[t]
    hs = np.histogram(df['pos_rad']*xshrink, np.arange(0, 2*np.pi*xshrink, 1e4*2*np.pi*xshrink/len(FASTA)))
    ax.plot((hs[1][:-1]+hs[1][1:])/2, y_pos+hs[0]/yscale, color=color, lw=0.5)
    ax.plot((hs[1][:-1]+hs[1][1:])/2, np.repeat(y_pos, hs[0].shape[0]), color='black', lw=0.5, zorder=10)

# plot the y scale
for i in range(0,11,2):
    ax.plot(np.array([1,1.002])*2*np.pi*xshrink, np.repeat(9+i/yscale, 2), color='black', lw=0.5)
    ax.text(1.002*2*np.pi*xshrink, 9+i/yscale, i, ha='left', va='bottom', rotation=(1-xshrink)*360, size=6)

legend_elements=[Line2D([0], [0], 
                        #marker="s", ms=10, color="#00000000", mec="#00000000", mfc=color_sv[i],
                        color=color_sv[i],
                        label=alias_sv[i]) for i in color_sv.keys()]
ax.legend(handles=legend_elements, loc='center', frameon=False)

plt.savefig("/Volumes/MacintoshHD/Dropbox/Jean_Talon/fig/Fig5A.svg")
plt.show()
plt.close()

In [ ]:
# Compute the distributions of distance to the nearest same-type SV for each pair of strains
strains_svim = strains[:4]

distances = {}

for (tig, sv), df in SV.groupby([0,'SVTYPE']):
    if sv in ["INS","DEL","DUP:TANDEM"]:

        for s1, s2 in itertools.combinations(strains_svim, 2):
            
            key = frozenset((s1, s2))
            if key not in distances:
                distances[key] = []
            
            sub1 = df.loc[df["strain"]==s1, "pos_abs"]
            sub2 = df.loc[df["strain"]==s2, "pos_abs"]
            
            if all([sub1.shape[0]>0, sub2.shape[0]>0]):
                # iterate over the s1 SVs and find the doistance to the closest same-type SV in s2
                for i in sub1.index:
                    value = min(abs(sub2-sub1.loc[i]))
                    distances[key].append([value, sv, s1, s2])
                # iterate over the s2 SVs and find the doistance to the closest same-type SV in s1
                for i in sub2.index:
                    value = min(abs(sub1-sub2.loc[i]))
                    distances[key].append([value, sv, s1, s2])

for key in distances:
    new = pd.DataFrame(distances[key], columns=['dist','sv','s1','s2'])
    new['dist_log'] = np.log10(new['dist'].replace(to_replace={0:1}).values)
    distances[key] = new

In [ ]:
# plot the distributions of distances and perform pairwise Mann-Whitney tests between distributions
strains_svim = strains[:4]
fig = plt.figure(figsize=[10,10])
gs = plt.GridSpec(nrows=3, ncols=3, hspace=0.3, wspace=0.3, left=0.12, bottom=0.12)
strain_plot_order = dict(zip(strains_svim, range(4)))

for (i,j) in itertools.combinations(strains_svim, 2):
    row_ax = strain_plot_order[j]-1
    col_ax = strain_plot_order[i]
    ax = fig.add_subplot(gs[row_ax, col_ax])
    
    data = distances[frozenset([i,j])]
    
    for sv, df in data.groupby('sv'):
        hs = np.histogram(df['dist_log'].values, np.arange(0,6,0.1))
        ax.plot(hs[1][:-1], hs[0], color=color_sv[sv])
        
        # plot median of the distribution as dotted line
        ax.axvline(np.median(df['dist_log'].values), color=color_sv[sv], linestyle=':', lw=2)
        
        ax.set_xlim(-0.2,6.2)
        ax.set_xticks(range(7))
        ax.set_ylim(0,250)
        
        
        if col_ax==0:
            ax.set_ylabel(strains_alias[j])
        else:
            ax.set_yticklabels([])
            
        if row_ax==2:
            ax.set_xlabel(strains_alias[i])
        else:
            ax.set_xticklabels([])
    if (row_ax, col_ax) == (0,0):
        ax.legend(handles=legend_elements[:3], loc=1, frameon=False)

MWU = {}
# reference distribution for distances is between Jean-Talon and Jean-Talon subset
ref_distro = distances[frozenset(['barcode11.ont','barcode11_sub.ont'])]
for sv, (row_ax, col_ax) in zip(["INS","DEL","DUP:TANDEM"], [(0,1),(0,2),(1,2)]):
    
    ax = fig.add_subplot(gs[row_ax, col_ax])
    
    ref_distro_sv = ref_distro.loc[ref_distro['sv']==sv, 'dist'].values
    mwu=[]
    
    for (i,j) in itertools.combinations(strains_svim, 2):
        distro1 = distances[frozenset([i,j])]
        distro1_sv = distro1.loc[distro1['sv']==sv, 'dist'].values
        s, p = mannwhitneyu(ref_distro_sv, distro1_sv, alternative='two-sided')
        
        mwu.append([s,p,i,j])
    mwu = pd.DataFrame(mwu, columns=['U','p','col','row'])
    # correct p-values for multiple testing
    mwu['pval_corr'] = np.log10(sm.stats.multipletests(mwu['p'].values, method='fdr_bh', alpha=0.05)[1])*(-1)
    #mwu['U_corr'] = mwu['U']/max(mwu['U'])
    # normalize U statistics by the maximum possible value
    mwu['U_norm'] = mwu['U']/(distro1_sv.shape[0]*ref_distro_sv.shape[0])
    MWU[sv] = mwu
    mwu_U = pd.pivot_table(mwu, index='row', columns='col', values='U_norm', 
                         aggfunc=lambda x: x).loc[strains_svim[1:], strains_svim[:-1]]
    mwu_p = pd.pivot_table(mwu, index='row', columns='col', values='pval_corr', 
                     aggfunc=lambda x: x).loc[strains_svim[1:], strains_svim[:-1]]
    
    sns.heatmap(mwu_U, ax=ax, cmap='viridis', cbar_kws={'format':'%.2f'}
                #, vmin=0, vmax=1
               )
    
    for (i,j) in itertools.combinations(strains_svim, 2):
        if mwu_p.loc[j,i] >= np.log10(0.05)*(-1):
            ax.scatter(strain_plot_order[i]+0.5, strain_plot_order[j]-0.5, color='red', marker='o', s=30)
    
    ax.text(2, 0.5, alias_sv[sv], ha='center', va='center')
    ax.set_xticklabels([strains_alias[s] for s in strains_svim[:-1]], size=6.5, rotation=0)
    ax.set_yticklabels([strains_alias[s] for s in strains_svim[1:]], size=6.5, rotation=90, va='center')
    ax.set_xlabel('')
    ax.set_ylabel('')
        

fig.text(0.5, 0.015, '$log_{10}$ distance to closest variant (bp)', size=16, ha='center', va='center')
fig.text(0.015, 0.5, 'Frequency', rotation=90, size=16, ha='center', va='center')

sns.despine()
plt.savefig('/Volumes/MacintoshHD/Dropbox/Jean_Talon/fig/Fig5b.svg')
plt.show()
plt.close()

# Analysis of translocations

In [ ]:
# generate alternate reference for S288c with 2 translocations
ASSEMBLY={}
with open('/Users/mathieu/mhenault_landrylab/Sequences/ref_genomes/S288C_pacbio/S288c.genome.fa') as file_in:
    for seq in SeqIO.parse(file_in, "fasta"):
        ASSEMBLY[seq.id] = seq
with open('/Users/mathieu/mhenault_landrylab/Sequences/ref_genomes/S288C_pacbio/S288c.all_feature.gff') as fi:
    gff = pd.DataFrame([i.split('\t') for i in fi.read().splitlines() if i[0]!='#']).astype({3:int,4:int})

gff['tig_len'] = [len(ASSEMBLY[i].seq) for i in gff[0].values]
gff['diff'] = gff['tig_len']-gff[4]
TY1 = gff.loc[(gff[2]=='TY1')]

In [ ]:
# a good subtelomeric TY1 is index 10528; chrVIII, pos 562-568k, minus strand.
# could be paired with index 16799; TY1 located half way on chrXIII
# Another translocation: 12275-4930 chrXIV-chrIV
translocations = [[10528,16799], [18938,4930], [12217,6196]]

In [ ]:
# generate the S288c assembly with translocations
NEW_ASSEMBLY = ASSEMBLY.copy()
for t in translocations:
    gff1_tig, gff2_tig = gff.loc[t, 0]
    gff1_pos, gff2_pos = gff.loc[t, 4]

    new_seq_1 = ASSEMBLY[gff1_tig].seq[:gff1_pos] + ASSEMBLY[gff2_tig].seq[gff2_pos:]
    new_seq_2 = ASSEMBLY[gff2_tig].seq[:gff2_pos] + ASSEMBLY[gff1_tig].seq[gff1_pos:]
    new_chrom_1 = SeqRecord.SeqRecord(seq=new_seq_1, id='_'.join([gff1_tig,gff2_tig]), description='')
    new_chrom_2 = SeqRecord.SeqRecord(seq=new_seq_2, id='_'.join([gff2_tig,gff1_tig]), description='')
    NEW_ASSEMBLY[new_chrom_1.id] = new_chrom_1
    NEW_ASSEMBLY[new_chrom_2.id] = new_chrom_2
    
    del NEW_ASSEMBLY[gff1_tig]
    del NEW_ASSEMBLY[gff2_tig]
    
#with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/data/S288c_genome_translocations.fasta','w') as fo:
#    SeqIO.write(NEW_ASSEMBLY.values(), fo, 'fasta')

In [ ]:
def aln_len_cigar(cigar):
    pattern = '[0-9]+[MIDNSHP=X]'
    aln_chunks = re.findall(pattern, cigar)
    seq = pd.Series([i[:-1] for i in aln_chunks], index=[i[-1] for i in aln_chunks]).astype(int)
    return(seq.loc[[i for i in ['M','I','S','=','X'] if i in seq.index]].sum())

In [ ]:
# function to parse sam files of reads with supplementary alignments
def sam_table(sam_path):
    SAM=[]
    with open(sam_path, 'r') as fi:
        for line in fi.read().splitlines():
            line_split = line.split('\t')
            read = line_split[0]
            flag = line_split[1]
            aln_len = aln_len_cigar(line_split[5])
            tig = line_split[2]
            aln_start = int(line_split[3])
            SAM.append([read, flag, tig, aln_start, aln_len])

    SAM = pd.DataFrame(SAM, columns=['read','flag','tig','aln_start','aln_len'])

    SAM['abs_start'] = SAM['aln_start']+fasta.loc[SAM['tig']].values
    SAM['abs_mid'] = SAM['abs_start']+0.5*SAM['aln_len']
    SAM['tig_order'] = tig_order.loc[SAM['tig']].values
    SAM = SAM.sort_values(by='abs_start')

    # keep only reads for which supplementary alignments are found on exactly two different chromosomes
    for read, df in SAM.groupby('read'):
        if len(set(df['tig']))==2:
            #
            df=df.sort_values(by='aln_len', ascending=False).iloc[:2].sort_values(by='tig_order')
            SAM.loc[df.index, 'read_no'] = (0,1)

    SAM = SAM.loc[np.invert(np.isnan(SAM['read_no']))]
    SAM_pivot = pd.pivot_table(SAM, values='abs_mid', index='read', columns='read_no')
    
    return(SAM, SAM_pivot)

In [ ]:
SAM = {}
SAM_pivot = {}

for s in strains:
    sam_path = f'/Volumes/MacintoshHD/Dropbox/Jean_Talon/translocations/{s}.supp.sam'
    sam, sam_pivot = sam_table(sam_path)
    SAM[s] = sam
    SAM_pivot[s] = sam_pivot

In [ ]:
#Export SAM
#with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/translocations/SAM.pkl','wb') as fo:
#    pkl.dump(SAM, fo)
#with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/translocations/SAM_pivot.pkl','wb') as fo:
#    pkl.dump(SAM_pivot, fo)

In [ ]:
#Import SAM
with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/translocations/SAM.pkl','rb') as fi:
    SAM = pkl.load(fi)
with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/translocations/SAM_pivot.pkl','rb') as fi:
    SAM_pivot = pkl.load(fi)

In [ ]:
# plot matrices of supplementary read mappings 
fig = plt.figure(figsize=[30,45])
gs = plt.GridSpec(ncols=18, nrows=3, left=0.1, bottom=0.1, right=0.98, top=0.98)
# define bin width
wdw=2e4

bins = pd.interval_range(start=1, end=(np.ceil(len(FASTA)/wdw)+1)*wdw, freq=wdw, closed='left')
ticks = fasta.copy()
ticks.loc['chr0'] = len(FASTA)
ticks_pos = (ticks/wdw).apply(lambda x: np.ceil(x))
labelpos = (ticks_pos[:-1].values+ticks_pos[1:].values)/2

for s, ax_idx in zip(strains, itertools.product(range(3), range(2))):

    c = cov[s]
    ax = fig.add_subplot(gs[ax_idx[0], ax_idx[1]*9:(ax_idx[1]+1)*9])

    matrix = pd.DataFrame(np.zeros([len(bins), len(bins)]), index=bins, columns=bins)

    SAM_bins = pd.concat([pd.cut(SAM_pivot[s][0], bins), pd.cut(SAM_pivot[s][1], bins)], axis=1)
    for (i,j), df in SAM_bins.groupby([0,1]):
        reads = df.index
        # approximate supplementary mapping depth by summing alignment lengths
        coverage_sum = SAM[s].loc[SAM[s]['read'].isin(reads), 'aln_len'].sum()
        matrix.loc[j,i] = coverage_sum/(wdw*c)
    matrix = matrix.values
    matrix[np.triu_indices_from(matrix)] = np.nan

    HM = ax.imshow(matrix, cmap='viridis', vmin=0, vmax=2)

    for i in ticks_pos:
        ax.plot((i,i), (0, len(bins)-1), color='white', lw=0.1)
        ax.plot((0, len(bins)-1), (i,i),  color='white', lw=0.1)

    for i, tig in zip(labelpos, tig_names):
        ax.text(i, i, s=tig, ha='left', va='bottom')

    # add circles around simulated translocations
    if s=='s288c_translocation.pb':
        for t in translocations:
            gff_sub = gff.loc[t].copy()
            gff_sub['pos_abs'] = gff_sub.apply(lambda x: fasta[x[0]]+np.mean(x[[3,4]]), axis=1)
            gff_sub = np.round(gff_sub['pos_abs'].sort_values()/2e4).astype(int)-1
            ax.scatter(gff_sub.iloc[0], gff_sub.iloc[1], marker='o', s=100, c=(0,0,0,0), edgecolors='red', linewidths=0.5)

    ax.set_xticks(ticks_pos)
    ax.set_xticklabels([f'{i:.3f}' for i in ticks/1e6], rotation=-90)
    ax.set_xlabel('Read segment 1 position (Mb)')

    ax.set_yticks(ticks_pos)
    ax.set_yticklabels([f'{i:.3f}' for i in ticks/1e6])
    ax.set_ylabel('Read segment 2 position (Mb)')

    ax.margins(0)

    ax.set_title(strains_alias[s], size=20)

    #plot the colorbar
    ax = fig.add_subplot(gs[ax_idx[0], ax_idx[1]*9+7])
    ax.axis('off')
    fig.colorbar(HM, ax=ax, shrink=2, orientation='vertical', drawedges=False, label='Approx. Normalized read depth')

sns.despine(trim=True)

plt.savefig('/Volumes/MacintoshHD/Dropbox/Jean_Talon/fig/FigS6.svg')
#plt.show()
plt.close()

In [ ]:
# plot the read length distributions for long-read datasets
#import read lenghts
rl={}
for s in strains[:6]:
    rl[s]=[]
    S='.'.join(s.split('.')[:-1])
    with open(f'/Volumes/MacintoshHD/Dropbox/Jean_Talon/data/{S}.size.fasta') as fi:
        for seq in SeqIO.parse(fi, 'fasta'):
            rl[s].append(len(seq.seq))

In [ ]:
#with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/Files_SX_SVs/rl.pkl', 'wb') as fo:
#    pkl.dump(rl, fo)

In [ ]:
fig, ax = plt.subplots(figsize=[6,6])
wdw=200
for s in rl.keys():
    h = np.histogram(rl[s], bins=np.arange(8e3,20e3+wdw,wdw), density=True)
    ax.plot(h[1][:-1]+wdw/2, h[0], label=strains_alias[s], lw=2)
    
plt.legend(frameon=False)
ax.set_xlabel('Read size (bp)')
ax.set_ylabel('Density')
sns.despine(trim=True)
plt.tight_layout()

plt.savefig('/Volumes/MacintoshHD/Dropbox/Jean_Talon/fig/FigS5.svg')
plt.show()
plt.close()

In [ ]:
# manually split the Jean-Talon assembly
JT={}
tig_order = []
with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/assemblies/Jean-Talon_reordered.fasta') as fi:
    for seq in SeqIO.parse(fi, 'fasta'):
        JT[seq.id] = seq
        tig_order.append(seq.id)

In [ ]:
ty2 = [93000+47862-1, 93000+53823]

JT_split = []
for tig in tig_order:
    if tig=='ctg6_pilon':
        new = SeqRecord.SeqRecord(JT['ctg6_pilon'].seq[:ty2[1]], id='ctg6.1_pilon')
        JT_split.append(new)
        new = SeqRecord.SeqRecord(JT['ctg6_pilon'].seq[ty2[0]:], id='ctg6.2_pilon')
        JT_split.append(new)
    else:
        JT_split.append(JT[tig])

In [ ]:
with open('/Volumes/MacintoshHD/Dropbox/Jean_Talon/assemblies/Jean-Talon_reordered_split.fasta', 'w') as fo:
    SeqIO.write(JT_split, fo, 'fasta')